In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
from sklearn.linear_model import LinearRegression
import random
import json
import requests
from bs4 import BeautifulSoup
from difflib import SequenceMatcher
import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
start_tiempo= time.process_time()
stemmer=LancasterStemmer();

with open('intents.json') as file:
  data=json.load(file)

words=[]
labels=[]
docs_x=[]
docs_y=[]

for intent in data['intents']:
  for pattern in intent['patterns']:
    wrds=nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
      labels.append(intent['tag'])

words=[stemmer.stem(w.lower()) for w in words if w not in "?"]
words=sorted(list(set(words)))
labels=sorted(labels)

In [ ]:
training=[]
output=[]

out_empty=[0 for _ in range(len(labels))]

for x,doc in enumerate(docs_x):
  bag=[]
  wrds=[stemmer.stem(w.lower()) for w in doc]
  for w in words:
    if w in wrds:
      bag.append(1)
    else:
      bag.append(0)
  output_row=out_empty[:]
  output_row[labels.index(docs_y[x])]=1

  training.append(bag)
  output.append(output_row)

training=np.array(training)
output=np.array(output)

In [ ]:
modelo=LinearRegression()

modelo.fit(training,output)

test=np.random.randint(0,2,size=(1,46))

In [ ]:
def bag_of_words(s,words):
  bag=[0 for _ in range (len(words))]

  s_words=nltk.word_tokenize(s)
  s_words=[stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i,w in enumerate(words):
      if w==se:
        bag[i]=1
  return np.array(bag)

In [ ]:
sujeto=['I', 'We', 'My name', 'It', 'Hello', 'You']
verbo=['am', 'have', 'does not', 'is', 'costs', 'sell', 'are', 'good to see', 'can', 'see', 'was']
predicado=['18', 'no idea', 'matter','a secret', 'Tim', '20', 'cookies', 'open at 8', 'closed at 20', 'you', 'help you', 'a good day', 'happy to see you', 'you later', 'a pleasure', 'sad']

def unirPalabras(vector):
  texto=''
  for palabra in vector:
    texto+=palabra+' '
  texto=texto[0:len(texto)-1]
  return texto

def poblacionInicial(poblacion):
  oraciones=[]
  for _ in range(poblacion):
    oraciones.append([random.choice(sujeto),random.choice(verbo),random.choice(predicado)])
  return oraciones

def determinarCoherencia(texto):
  texto=unirPalabras(texto)
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36"}
  URL     = 'https://www.google.com/search?q="'+texto+'"'
  result = requests.get(URL, headers=headers)    

  soup = BeautifulSoup(result.content, 'html.parser')

  total_results_text = soup.find("div", {"id": "result-stats"}).find(text=True, recursive=False) # this will give you the outer text which is like 'About 1,410,000,000 results'
  results_num = ''.join([num for num in total_results_text if num.isdigit()]) # now will clean it up and remove all the characters that are not a number .
  print(results_num)
  return int(results_num)

def mutar_poblacion(oraciones):
  oraciones_final=[]
  for oracion in oraciones:
    oraciones_final.append(mutar_individuo(oracion))
  return oraciones_final

def mutar_individuo(oracion):
  prob=1/len(oracion)

  if (prob>random.uniform(0,1)):
    oracion[0]=random.choice(sujeto)
  if (prob>random.uniform(0,1)):
    oracion[1]=random.choice(verbo)
  if (prob>random.uniform(0,1)):
    oracion[2]=random.choice(predicado)

  return oracion
  #Funcion para guardar el sujeto con mejor fitness
def asignarMejor(actual,nuevo,palabra_actual,palabra_nueva):
  if actual>nuevo:
    return actual,palabra_actual
  else:
    return nuevo,palabra_nueva

def determinarFitness(respuestas,oracion):
  fitness=[]
  oracion=unirPalabras(oracion)
  for respuesta in respuestas:
    ratio = SequenceMatcher(None, oracion, respuesta).ratio()
    fitness.append(ratio)
  return max(fitness)

def respuestasAlgoritmoGenetico(respuestas, poblacion):
  iteraciones=0
  while (True):
    oraciones=poblacionInicial(poblacion)
    mejor_fitness=0
    mejor_oracion=[]
    while mejor_fitness<0.9:
      fitness=[]
      oraciones=mutar_poblacion(oraciones)
      #obtener fitness de cada sujeto:
      for oracion in oraciones:
        fitness.append(determinarFitness(respuestas,oracion))
      indices=np.argsort(fitness)[::-1]
      mejor_fitness,mejor_oracion=asignarMejor(mejor_fitness,fitness[indices[0]],mejor_oracion,oraciones[indices[0]])
    iteraciones+=1
    if (determinarCoherencia(mejor_oracion)>2000000 or iteraciones>=5):      
      break
  return unirPalabras(mejor_oracion)

In [ ]:
def chat():
  print('Start talking with the bot (type quit to stop)!')
  while True:
    inp=input('You: ')
    if inp.lower()=="quit":
      break
    results=modelo.predict([bag_of_words(inp,words)])[0]
    results_index=np.argmax(results)
    tag=labels[results_index]

    if results[results_index]>0.7:
      for tg in data['intents']:
        if tg['tag']==tag:
          responses=tg['responses']
      print(respuestasAlgoritmoGenetico(responses,100))
    else:
      print('Try again.')
    print("Tiempo ejecucion = "+ str(time.process_time() - start_tiempo))


In [ ]:
chat()

Start talking with the bot (type quit to stop)!
2790000
I was happy to see you
Tiempo ejecucion = 0.9987652850000002
